In [17]:
!pip install python-Levenshtein

Looking in indexes: https://smartiproxy.mgmt.netflix.net/pypi


In [20]:
from Levenshtein import distance

In [19]:
data = 'data/geo880_train600.tsv'

In [ ]:
for 

In [30]:
from collections import Counter

In [31]:
src_tgt = [line.strip().split('\t') for line in open(data)]

Counter([
    (x, y)
    for src, tgt in src_tgt
    for i, (x, y) in enumerate(zip(src.split(), tgt.split()))
    if distance(x, y) <= 2 and len(x) >= 3 and len(y) >= 3
])

Counter({('largest', '_largest'): 9,
         ('smallest', '_smallest'): 4,
         ('highest', '_highest'): 2,
         ('longest', '_longest'): 2,
         ('most', '_most'): 5,
         ('capital', '_capital'): 1,
         ('states', '_state'): 3,
         ('state', '_state'): 1})

In [41]:
friends = set()
friends.union({1, 2, 3})

{1, 2, 3}

In [58]:
friends = []

window_size = 5
distance_threshold = 3
token_size_threshold = 3

for src, tgt in src_tgt:
    src_tokens = src.split()
    tgt_tokens = tgt.split()
    
    for i, src_token in enumerate(src_tokens):
        if len(src_token) >= token_size_threshold:
            friends += [
                (src_token, tgt_tokens[i + j])
                for j in range(-window_size, window_size + 1)
                if (0 <= i + j < len(tgt_tokens) and len(tgt_tokens[i + j]) >= token_size_threshold and distance(tgt_tokens[i + j], src_token) <= distance_threshold)
            ]

In [59]:
Counter(friends)

Counter({('highest', '_highest'): 26,
         ('state', '_state'): 62,
         ('mountain', '_mountain'): 4,
         ('capital', '_capital'): 25,
         ('populations', '_population'): 7,
         ('name', '_lake'): 1,
         ('lakes', '_lake'): 4,
         ('states', '_state'): 96,
         ('population', '_population'): 37,
         ('rivers', '_river'): 31,
         ('lowest', '_lowest'): 14,
         ('capitals', '_capital'): 4,
         ('size', '_size'): 5,
         ('shortest', '_shortest'): 9,
         ('river', '_river'): 32,
         ('major', '_major'): 27,
         ('most', '_most'): 13,
         ('long', '_len'): 7,
         ('the', '_len'): 18,
         ('area', '_area'): 15,
         ('longest', '_longest'): 21,
         ('smallest', '_smallest'): 21,
         ('largest', '_largest'): 42,
         ('city', '_city'): 26,
         ('over', '_river'): 1,
         ('are', '_lake'): 1,
         ('density', '_density'): 8,
         ('mountains', '_mountain'): 1,
       

In [61]:
tuples = [
    (x, y)
    for src, tgt in src_tgt
    for i, (x, y) in enumerate(zip(src.split(), tgt.split()))
    # if distance(x, y) <= 2 and len(x) >= 3 and len(y) >= 3
]

In [62]:
from collections import defaultdict

In [73]:
cnt = defaultdict(lambda: defaultdict(int))

In [74]:
for src, tgt in tuples:
    if len(src) >= 3 and len(tgt) >= 3:
        cnt[tgt][src] += 1

In [81]:
for src in cnt['_city']:
    print(src)

the
are
border
pennsylvania
usa
virginia
texas
citizens
borders
california
population


In [ ]:
{
    ''
}

{'_answer': {'what': 384, 'which': 53, 'how': 111, 'where': 17},
 '_largest': {'state': 13, 'city': 19}}

In [102]:
from nltk.corpus import stopwords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [108]:
cnt = defaultdict(lambda: defaultdict(int))
cnt_tgt = defaultdict(int)

window_size = 3
# distance_threshold = 3
token_size_threshold = 3
support = 0.3
support_abs = 2
stop_words = set(stopwords.words('english'))


for src, tgt in src_tgt:
    src_tokens = src.split()
    tgt_tokens = tgt.split()
    
    for tgt_token in tgt_tokens:
        cnt_tgt[tgt_token] += 1
    
    for i, src_token in enumerate(src_tokens):
        if len(src_token) >= token_size_threshold and src_token not in stop_words:
            for j in range(-window_size, window_size + 1):
                if (0 <= i + j < len(tgt_tokens) and len(tgt_tokens[i + j]) >= token_size_threshold and tgt_tokens[i + j] not in stop_words):
                    cnt[tgt_tokens[i + j]][src_token] += 1
                    
{
    tgt: {src: cnt[tgt][src] for src in cnt[tgt] if cnt[tgt][src] >= support * cnt_tgt[tgt] and cnt[tgt][src] >= support_abs}
    for tgt in cnt
    if len({src for src in cnt[tgt] if cnt[tgt][src] >= support * cnt_tgt[tgt] and cnt[tgt][src] >= support_abs}) >= 2
}

{'_highest': {'highest': 26, 'point': 18},
 '_lake': {'lakes': 3, 'states': 2},
 '_lowest': {'lowest': 14, 'point': 12},
 '_count': {'many': 59, 'states': 32, 'cities': 18, 'rivers': 19},
 '_shortest': {'shortest': 9, 'river': 9},
 '_major': {'major': 24, 'cities': 20},
 '_most': {'states': 7, 'state': 7},
 '_elevation': {'highest': 9, 'point': 7},
 '_len': {'river': 9, 'length': 4},
 '_longest': {'longest': 21, 'river': 21},
 '_smallest': {'smallest': 20, 'state': 15, 'population': 12},
 '_largest': {'state': 35, 'largest': 39},
 '_density': {'population': 10, 'density': 8},
 '_sum': {'population': 2, 'states': 2},
 '_fewest': {'state': 2, 'borders': 2, 'least': 2, 'states': 2}}